# Project Part 3

[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/Matthew-Bustamante/CS39AA-Project-Cyberbullying/blob/main/project_part3.ipynb)


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Matthew-Bustamante/CS39AA-Project-Cyberbullying/blob/main/project_part3.ipynb)


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch.nn.functional as F
import torch.cuda
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
input_data_path = 'https://raw.githubusercontent.com/Matthew-Bustamante/CS39AA-Project-Cyberbullying/main/CyberBullying_Comments_Dataset.csv'
df = pd.read_csv(input_data_path)
df.head()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset, load_metric

In [ ]:
MODEL_NAME = "bert-base-cased"
MAX_LENGTH=50 

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3, max_length=MAX_LENGTH, output_attentions=False, output_hidden_states=False)

Creating Vocabulary for y(dependent variable).  Note that 0 = non-cyberbullying and 1 = cyberbullying

In [ ]:
classes = df.CB_Label.unique().tolist()
class_tok2idx = dict((v, k) for k, v in enumerate(classes))
class_idx2tok = dict((k, v) for k, v in enumerate(classes))
print(class_tok2idx)
print(class_idx2tok)

Creating a new column with these labels which will be the y that is used

In [ ]:
df['label'] = df['CB_Label'].apply(lambda x: class_tok2idx[x])
df.head()

In [ ]:

sequence_0 = "Damn  hope you getz better soon !!!"
seq0_tokens = tokenizer(sequence_0, return_tensors="pt")
print(f"number of tokens in seq0 is {len(seq0_tokens['input_ids'].flatten())}")
print(seq0_tokens)
F.softmax(model(**seq0_tokens).logits, dim=1)


In [ ]:
sequence_1 = "Shakespeare nerd!"
seq0_tokens = tokenizer(sequence_1, return_tensors="pt")
print(f"number of tokens in seq1 is {len(seq0_tokens['input_ids'].flatten())}")
print(seq0_tokens)
F.softmax(model(**seq0_tokens).logits, dim=1)

In [ ]:
sequence_2 = "Is there ever a day that mattresses are not on sale?"
seq0_tokens = tokenizer(sequence_2, return_tensors="pt")
print(f"number of tokens in seq2 is {len(seq0_tokens['input_ids'].flatten())}")
print(seq0_tokens)
F.softmax(model(**seq0_tokens).logits, dim=1)

In [ ]:
sequence_3 = "Dood! You just justified your MBA. I pity the sorry ass of that downtrodden employee whose manager you'll soon become."
seq0_tokens = tokenizer(sequence_3, return_tensors="pt")
print(f"number of tokens in seq3 is {len(seq0_tokens['input_ids'].flatten())}")
print(seq0_tokens)
F.softmax(model(**seq0_tokens).logits, dim=1)

In [ ]:
ds_raw = Dataset.from_pandas(df[['label', 'Text']])

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)

ds = ds_raw.map(tokenize_function, batched=True)

In [ ]:
ds[0]

In [ ]:
ds = ds.shuffle(seed=42)
ds[0]

In [ ]:
train_prop = 0.85
ds_train = ds.select(range(int(len(ds)*train_prop)))
ds_eval = ds.select(range(int(len(ds)*train_prop), len(ds)))

In [ ]:
print(f"len(ds_train) = {len(ds_train)}")
print(f"len(ds_eval) = {len(ds_eval)}")

In [ ]:
import os 
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(num_train_epochs=10,
                                  do_train=True,
                                  report_to=None,
                                  output_dir="/kaggle/working",
                                  evaluation_strategy="steps",
                                  eval_steps=200,
                                  learning_rate=1e-5,
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=32)

trainer = Trainer(model = model, 
                  args = training_args,
                  train_dataset = ds_train, 
                  eval_dataset = ds_eval,
                  compute_metrics = compute_metrics,
)

In [ ]:
if torch.cuda.is_available():
    device = "cuda:0"
    print("Using GPU")
else:
    device = "cpu"

In [ ]:
model.to(device)

In [ ]:
torch.set_grad_enabled(True)
trainer.train()
trainer.evaluate()

In [ ]:
df_test = pd.read_csv("https://raw.githubusercontent.com/Matthew-Bustamante/CS39AA-Project-Cyberbullying/main/CyberBullying_Comments_Dataset.csv")
df_test.head()

In [ ]:
ds_test_raw = Dataset.from_pandas(df_test)
ds_test_raw[0]

In [ ]:
ds_test = ds_test_raw.map(tokenize_function, batched=True)

In [ ]:
preds = trainer.predict(test_dataset=ds_test)

In [ ]:
test_preds = np.apply_along_axis(np.argmax, 1, preds.predictions)

In [ ]:
df_test['preds'] = test_preds.tolist()
df_test['CB_Label'] = df_test['preds'].apply(lambda x: class_idx2tok[x])
df_test.head()